# Import

In [1]:
import os
import sys
import json
import pandas as pd
import numpy as np
import torch
import pickle
import pathSetting
import Parameter as P

sys.path.append("../")
sys.path.append("../../")
sys.path.append("../../..")

from KETIToolDL.CLUSTTool.common import p1_integratedDataSaving as p1
import getIntegrationData as gID

c:\Users\korea\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. Data Split&Transform&Integration
## 1-1. 3D Array Data Read

In [2]:
# 1. UCI HAR Data Read
folderAddress = './data/'
dataset_train_x = pickle.load(open(folderAddress + 'x_train.pkl', 'rb'))
dataset_train_y = pickle.load(open(folderAddress + 'y_train.pkl', 'rb'))
dataset_test_x = pickle.load(open(folderAddress + 'x_test.pkl', 'rb'))
dataset_test_y = pickle.load(open(folderAddress + 'y_test.pkl', 'rb'))

In [3]:
# 1-1. train shape 확인
dataset_train_x.shape, dataset_train_y.shape

((7352, 9, 128), (7352,))

In [4]:
# 1-2. test shape 확인
dataset_test_x.shape, dataset_test_y.shape

((2947, 9, 128), (2947,))

## 1-2. Basic Setting

In [5]:
# 2. Basic Setting

dataModeList = ['trainX', 'trainy', 'testX', 'testy']
transformModeList = ['splitAndTransform', 'onlyTransform']

# 2-1. 변형 및 통합을 원하는 데이터 모드 선택
dataMode = dataModeList[0]

# 2-2. 분할 및 변형 방법 모드 선택 : train/test 둘 다 X데이터일때만 선택
if (dataMode == "trainX") | (dataMode == "testX"):
    transformMode = transformModeList[0]

transformMode = transformModeList[0]
# transformMode = transformModeList[1]

# 2-3. set parameter
trainStartTime = "2022-01-01"
testStartTime = "2022-02-01"
splitNum = None
transformFreqList = None
integration_method = "meta" #ML/meta
window_size = None # ML 통합 시 필요한것으로 rnn ae 학습 시 활용되는 window size. 통합 데이터 하나의 길이
sliding_size = None # rnn

intParam = P.setParameter(integration_method, window_size, sliding_size)

cuda is available.


In [6]:
# 2-4. 선택한 모드에 따라 parameter 설정
if dataMode == 'trainX':
    originalDataset = dataset_train_x.copy() 
    startTime = trainStartTime
    if transformMode == 'splitAndTransform':
        splitNum = 3
        transformFreqList = ['1S', '2S', '4S']
    elif transformMode == 'onlyTransform':
        splitNum = 1
        transformFreqList = ['4S']
        
elif dataMode == 'trainy':
    originalDataset = dataset_train_y.copy() 
    startTime = trainStartTime
    
elif dataMode == 'testX':
    originalDataset = dataset_test_x.copy() 
    startTime = testStartTime
    if transformMode == 'splitAndTransform':
        splitNum = 3
        transformFreqList = ['1S', '2S', '4S']
    elif transformMode == 'onlyTransform':
        splitNum = 1
        transformFreqList = ['4S']

elif dataMode == 'testy':
    originalDataset = dataset_test_y.copy() 
    startTime = testStartTime

## 1-3. Get IntegrationData & window

In [7]:
# 3. 분할 및 변형, 통합 데이터 생성
# 3-1. 모드에 따라 설정한 기본 parameter와 P.parameter를 입력
gid = gID.getIntegrationData(originalDataset, startTime, splitNum, transformFreqList, **intParam)
intDataSet, window = gid.getIntegrationDataByMethod(mode = dataMode)

dataset shape :  (7352, 9, 128)
array num :  0
......................
split num :  0
split data shape :  (128, 3)
------
split num :  1
split data shape :  (64, 3)
------
split num :  2
split data shape :  (32, 3)
------
array num :  1
......................
split num :  0
split data shape :  (128, 3)
------
split num :  1
split data shape :  (64, 3)
------
split num :  2
split data shape :  (32, 3)
------
array num :  2
......................
split num :  0
split data shape :  (128, 3)
------
split num :  1
split data shape :  (64, 3)
------
split num :  2
split data shape :  (32, 3)
------
array num :  3
......................
split num :  0
split data shape :  (128, 3)
------
split num :  1
split data shape :  (64, 3)
------
split num :  2
split data shape :  (32, 3)
------
array num :  4
......................
split num :  0
split data shape :  (128, 3)
------
split num :  1
split data shape :  (64, 3)
------
split num :  2
split data shape :  (32, 3)
------
array num :  5
........

In [12]:
# 3-2. 통합 데이터
intDataSet

,0,1,2,3,4,5,6,7,8
2022-01-01 00:00:00,0.006165,0.008441,0.052216,0.037498,0.060221,0.016044,1.018889,-0.124756,0.104319
2022-01-01 00:00:04,0.006457,0.004794,0.041710,0.048204,0.057034,0.006305,1.019416,-0.126370,0.105895
2022-01-01 00:00:08,0.005627,0.005586,0.022976,0.047073,0.051204,0.013594,1.018831,-0.123677,0.098265
2022-01-01 00:00:12,0.005979,0.007366,0.012608,0.030755,0.047718,0.014785,1.019443,-0.120155,0.097940
2022-01-01 00:00:16,0.006535,0.003807,0.003966,0.025077,0.053256,0.008703,1.020278,-0.122136,0.098213
...,...,...,...,...,...,...,...,...,...
2022-01-11 21:23:56,0.071711,0.220117,-0.142116,0.394050,-0.394212,0.388297,1.046553,-0.001916,-0.161062
2022-01-11 21:24:00,0.324464,0.105589,-0.295573,0.474012,-0.810454,0.271096,1.296199,-0.113168,-0.313809
2022-01-11 21:24:04,0.433770,-0.128383,-0.390633,0.487955,-0.300210,0.320099,1.403185,-0.342385,-0.410155
2022-01-11 21:24:08,0.048243,-0.278170,-0.240508,0.722453,0.551468,-0.032017,1.017119,-0.486515,-0.263757


In [14]:
np.array(intDataSet).reshape(7352, 32, 9).shape

(7352, 32, 9)

In [15]:
temp = np.array(intDataSet).reshape(7352, 32, 9)

In [16]:
pd.DataFrame(temp[0])

,0,1,2,3,4,5,6,7,8
0,0.006165,0.008441,0.052216,0.037498,0.060221,0.016044,1.018889,-0.124756,0.104319
1,0.006457,0.004794,0.041710,0.048204,0.057034,0.006305,1.019416,-0.126370,0.105895
2,0.005627,0.005586,0.022976,0.047073,0.051204,0.013594,1.018831,-0.123677,0.098265
3,0.005979,0.007366,0.012608,0.030755,0.047718,0.014785,1.019443,-0.120155,0.097940
4,0.006535,0.003807,0.003966,0.025077,0.053256,0.008703,1.020278,-0.122136,0.098213
5,0.006558,-0.000547,-0.000427,0.035860,0.048473,0.006038,1.020600,-0.125081,0.101576
6,0.005299,-0.002261,-0.007618,0.041039,0.037255,0.008362,1.019663,-0.125560,0.100987
7,0.004188,-0.000994,-0.017548,0.039796,0.033185,0.010434,1.018897,-0.123240,0.096534
8,0.004100,-0.004802,-0.019989,0.047000,0.027183,0.014493,1.019170,-0.126178,0.098487
9,0.004353,0.001690,-0.014308,0.042766,0.027846,0.014633,1.019793,-0.118992,0.107532


In [64]:
# timeElapsed 설정
timeElapsed = gid.timeElapsed

In [65]:
window

1

## 1-4. Data Save & Json Save

In [66]:
# 4. Data&Json Save
# 4-1. Json Info 설정
datasetName = 'ActionPatternData'
dataName = dataMode+'_'+datasetName+'_Split'+str(splitNum)

# intParam = P.setParameter(integration_method, window_size, sliding_size)

processParam = intParam["processParam"]
# dataInfo = P.parameter["dataInfo"]
integration_freq_sec = intParam["integration_freq_sec"]
endTime = str(intDataSet.index[-1])
cleanParam = intParam["cleanParam"]
DataSaveMode = intParam["DataSaveMode"]

dataInfo = {
    "windows" : window,
    "dataSize" : str(intDataSet.size),
    "timeElapsed" : timeElapsed
}

In [67]:
# 4-2. dataName 확인
dataName

'trainy_ActionPatternData_SplitNone'

In [68]:
data = intDataSet.copy()
# dataName = integration_method+"_"+dataMode+'_'+datasetName+'_Split'+str(splitNum)
# endTime = str(data.index[-1])

print(dataName)
#print(data.info())

# 4-3. Data Save
fileName = p1.saveCSVData(pathSetting.dataFolderPath, dataName, data)

# 4-4. Json Save
p1.saveJsonMeta(pathSetting.DataMetaPath, dataName, processParam, dataInfo,integration_freq_sec, 
                startTime, endTime, cleanParam, DataSaveMode)

trainy_ActionPatternData_SplitNone
